In [3]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm import tqdm

from sklearn.preprocessing import QuantileTransformer, PowerTransformer
from sklearn.metrics import make_scorer, log_loss
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

RANDOM_SEED = 777
np.random.seed(RANDOM_SEED)
np.random.SeedSequence(RANDOM_SEED)
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)

In [4]:
metadata = pd.read_csv('../input/is-this-crazy/metadata.csv')
sup_metadata = pd.read_csv('../input/is-this-crazy/supplemental_metadata.csv')
train_labels = pd.read_csv('../input/is-this-crazy/train_labels.csv', index_col="sample_id")
val_labels = pd.read_csv('../input/is-this-crazy/val_labels.csv',index_col='sample_id')
# targets = pd.read_csv('targets.csv',index_col='sample_id')
submission_format = pd.read_csv('../input/is-this-crazy/submission_format.csv')

In [5]:
targets = pd.concat([train_labels,val_labels],axis=0)
targets.shape

(1059, 10)

In [6]:
sam_id = metadata[metadata.instrument_type == 'sam_testbed'].index

In [7]:
# only change the data sets here...
train_path = "../input/ms-gen-train-test-data/train_temp_500.csv"
test_path =  '../input/ms-gen-train-test-data/test_temp_500.csv'
val_path = '../input/ms-gen-train-test-data/val_temp_500.csv'

In [8]:
oof_train_path = 'oof_train_temp_500.csv'
oof_test_path = 'oof_test_temp_500.csv'

In [9]:
%%time
test_100 = pd.read_csv(test_path,index_col=0,header=[0,1])
train_100 = pd.read_csv(train_path,index_col=0,header=[0,1])
val_100 = pd.read_csv(val_path,index_col=0,header=[0,1])

val_100.index = metadata[metadata.split == 'val'].sample_id

val_100.sort_index(inplace=True,axis=1)
test_100.sort_index(inplace=True,axis=1)
train_100.sort_index(inplace=True,axis=1)

CPU times: user 190 ms, sys: 38.7 ms, total: 229 ms
Wall time: 612 ms


In [10]:
# train_100.drop(columns=[('sample_id_encoded', 'Unnamed: 1601_level_1')],axis=1,inplace=True)

In [11]:
train_set_100 = pd.concat([train_100,val_100],axis=0)
test_set_100 = pd.concat([val_100,test_100],axis=0)
train_set_100.shape, test_set_100.shape

((1059, 400), (804, 400))

In [12]:
skf = StratifiedKFold(n_splits=20,random_state=RANDOM_SEED,shuffle=True)
log_loss_scorer = make_scorer(log_loss,needs_proba=True)

In [13]:
phy = 'phyllosilicate'
bas = 'basalt'
lr = LogisticRegression(solver='liblinear',C=10,random_state=RANDOM_SEED,penalty='l1')
lr

LogisticRegression(C=10, penalty='l1', random_state=777, solver='liblinear')

In [14]:
def get_coefs(X,y,c):
    lr = LogisticRegression(solver='liblinear',C=c,random_state=RANDOM_SEED,penalty='l1')
    print(cross_val_score(lr,X,y,cv=skf,scoring=log_loss_scorer).mean())
    lr.fit(X,y)
    # print(log_loss(y,lr.predict_proba(X)))
    coefs =  lr.coef_
    coefs_list  = []
    for id,wt in enumerate(tqdm(coefs[0])):
        if wt != 0:
            coefs_list.append(id)
    print(len(coefs_list))
    return coefs_list

In [15]:
def get_mz(mz,df):
    ''' returns list of columns for given mz'''
    cols_list = []
    for col in df.columns:
        if col[0] == str(mz):
            cols_list.append(col)
    return cols_list

In [16]:
def get_total_abundance(df):
    ''' returns total abundance for each sample'''
    tot_abund_df = pd.DataFrame(index=df.index)
    tot_abund_df['tot_abund'] = df.sum(axis=1)
    for mz in tqdm(range(0,100)):
        tot_abund_df[f"tot_abund_{mz}"] = df[get_mz(mz,df)].sum(axis=1)
    print(tot_abund_df.shape)
    return tot_abund_df

In [17]:
def get_grads(df,bin_size=5):
    ''' returns gradients for each mz'''
    grad_df = pd.DataFrame(index=df.index)
    for mz in tqdm(range(0,100)):
        df_mz = df.loc[:,get_mz(mz,df)]
        # df_mz_col1 = df_mz.iloc[:,0].values
        df_mz = df_mz.diff(axis=1)
        # df_mz.iloc[:,0] = df_mz_col1
        df_mz.dropna(axis='columns',inplace=True)
        # df_mz = df_mz/bin_size
        df_mz.columns = [col[0]+ '_' + col[1] + '_' + 'grad' for col in df_mz.columns]
        grad_df = pd.concat([grad_df,df_mz],axis=1)
    print(grad_df.shape)

    return  grad_df

In [18]:
def get_mean_std(df):
    ''' returns mean and std for each mz'''
    mean_std_df = pd.DataFrame(index=df.index)
    for mz in tqdm(range(0,100)):
        df_mz = df.loc[:,get_mz(mz,df)]
        df_mz[f'{mz}_mean'] = df_mz.mean(axis=1)
        df_mz[f'{mz}_std'] = df_mz.std(axis=1)
        mean_std_df = pd.concat([mean_std_df,df_mz],axis=1)
    print(mean_std_df.shape)
    return mean_std_df

In [19]:
# def add_fts(df):
#     tot_abund_df = get_total_abundance(df)
#     grad_df = get_grads(df)
#     mean_std_df = get_mean_std(df)
#     new_df = pd.concat([tot_abund_df,grad_df,mean_std_df],axis=1)
#     return new_df

In [20]:
train_set_100.shape, test_set_100.shape

((1059, 400), (804, 400))

In [21]:
assert (train_set_100.columns == test_set_100.columns).all()

In [22]:
t_100 = get_total_abundance(train_set_100)
g_100 = get_grads(train_set_100)
m_100 = get_mean_std(train_set_100)

100%|██████████| 100/100 [00:00<00:00, 313.74it/s]


(1059, 101)


100%|██████████| 100/100 [00:00<00:00, 135.29it/s]


(1059, 300)


100%|██████████| 100/100 [00:00<00:00, 150.69it/s]

(1059, 600)


In [23]:
t_100_test = get_total_abundance(test_set_100)
g_100_test = get_grads(test_set_100)
m_100_test = get_mean_std(test_set_100)


100%|██████████| 100/100 [00:00<00:00, 309.37it/s]


(804, 101)


100%|██████████| 100/100 [00:00<00:00, 128.76it/s]


(804, 300)


100%|██████████| 100/100 [00:00<00:00, 153.06it/s]

(804, 600)


In [24]:
t_100_test.sort_index(inplace=True,axis=1)
t_100.sort_index(inplace=True,axis=1)

m_100_test.sort_index(inplace=True,axis=1)
m_100.sort_index(inplace=True,axis=1)

g_100_test.sort_index(inplace=True,axis=1)
g_100.sort_index(inplace=True,axis=1)

In [25]:
assert (t_100.columns == t_100_test.columns).all()
assert (g_100.columns == g_100_test.columns).all()
assert (m_100.columns == m_100_test.columns).all()

In [26]:
ntrain = 1059
ntest = 804
NFOLDS = 20

In [27]:
oof_train_df = pd.DataFrame(index=metadata[metadata.split != 'test'].index, columns=['sample_id',*targets.columns])
oof_test_df = pd.DataFrame(index=metadata[metadata.split != 'train'].index, columns=['sample_id',*targets.columns])

In [28]:
agg_loss = []
for target in targets.columns:
    print('TARGET ',target)
    imp_fts = {}
    
    #  get all imp coefs for the target
    t_100_coefs = get_coefs(t_100,targets[target],1)
    g_100_coefs = get_coefs(g_100,targets[target],5)
    m_100_coefs = get_coefs(m_100,targets[target],10)
    o_100_coefs = get_coefs(train_set_100,targets[target],10)
    
    imp_fts['o_100_coefs'] = o_100_coefs
    imp_fts['t_100_coefs'] = t_100_coefs
    imp_fts['g_100_coefs'] = g_100_coefs
    imp_fts['m_100_coefs'] = m_100_coefs

    # get all imp_dfs and concat them
    train_imp_ = train_set_100.iloc[:,o_100_coefs]
    t_100_imp_ = t_100.iloc[:,t_100_coefs]
    g_100_imp_ = g_100.iloc[:,g_100_coefs]
    m_100_imp_ = m_100.iloc[:,m_100_coefs]

    print(train_imp_.shape,t_100_imp_.shape,g_100_imp_.shape,m_100_imp_.shape)
    X = pd.concat([train_imp_,t_100_imp_,g_100_imp_,m_100_imp_],axis=1)
    print(X.shape)

    # same goes for test set
    test_imp_ = test_set_100.iloc[:,o_100_coefs]
    t_100_imp_test = t_100_test.iloc[:,t_100_coefs]
    g_100_imp_test = g_100_test.iloc[:,g_100_coefs]
    m_100_imp_test = m_100_test.iloc[:,m_100_coefs]

    print(test_imp_.shape,t_100_imp_test.shape,g_100_imp_test.shape,m_100_imp_test.shape)
    
    X_test = pd.concat([test_imp_, t_100_imp_test, g_100_imp_test, m_100_imp_test],axis=1)
    print(X_test.shape)
    
    #  add feature
    X['is_sam'] = 0
    X.loc[sam_id[:12],'is_sam'] = 1
    X_test['is_sam'] = 0
    X_test.loc[sam_id[12:],'is_sam'] = 1
    
    
    assert (X.columns == X_test.columns).all()

    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))
    
    Y = targets[target]

    cb_clf = CatBoostClassifier(verbose=999,random_seed=RANDOM_SEED,n_estimators=5000)
    cv_loss = []
    for fold, (train_id, test_id) in enumerate(tqdm(skf.split(X,Y))):

        X_train, Y_train = X.iloc[train_id,], Y.iloc[train_id,]
        X_val, Y_val = X.iloc[test_id,], Y.iloc[test_id,]
        #  ohh man unwanted semi colon in iloc throws an error...
        
        # train model  and pred on oof folds
        cb_clf.fit(X_train.values,Y_train)
        y_preds = cb_clf.predict_proba(X_val.values)[:,1]
        fold_loss = log_loss(Y_val,y_preds)
        print(f'fold Loss {fold}: ',fold_loss)
        cv_loss.append(fold_loss)

        oof_train[test_id] = y_preds
        oof_test_skf[fold,:] = cb_clf.predict_proba(X_test.values)[:,1]
        
        print("---------------------------------------------------------------------------")
        
    print('Training Loss:', log_loss(Y,cb_clf.predict_proba(X.values)[:,1]))
    
    oof_test[:] = oof_test_skf.mean(axis=0)
    oof_test_df[target] = oof_test
    oof_train_df[target] = oof_train

    print(f'Loss for {target}:')
    print(np.array(cv_loss).mean(),np.array(cv_loss).std())
    agg_loss.append(np.array(cv_loss).mean())
    print("==================================================================================================")

print('AGG Loss:',np.array(agg_loss))
print('AGG Loss mean:',np.array(agg_loss).mean())

TARGET  basalt
0.31760258901095595


100%|██████████| 101/101 [00:00<00:00, 497212.09it/s]

9


0.33313037445516736


100%|██████████| 300/300 [00:00<00:00, 353949.70it/s]

...

4995:	learn: 0.0051382	total: 41s	remaining: 32.9ms
4999:	learn: 0.0051314	total: 41.1s	remaining: 0us
fold Loss 17:  0.0875018835664118
---------------------------------------------------------------------------
Learning rate set to 0.002361
0:	learn: 0.6894356	total: 12.3ms	remaining: 1m 1s
999:	learn: 0.0412264	total: 8.39s	remaining: 33.6s
1998:	learn: 0.0170323	total: 16.7s	remaining: 25.1s
2997:	learn: 0.0094540	total: 24.8s	remaining: 16.5s
3996:	learn: 0.0058817	total: 33s	remaining: 8.27s


19it [13:06, 41.51s/it]

4995:	learn: 0.0042083	total: 41.4s	remaining: 33.1ms
4999:	learn: 0.0042043	total: 41.4s	remaining: 0us
fold Loss 18:  0.06190592367671603
---------------------------------------------------------------------------
Learning rate set to 0.002362
0:	learn: 0.6894617	total: 16.2ms	remaining: 1m 21s
999:	learn: 0.0396769	total: 8.22s	remaining: 32.9s
1998:	learn: 0.0166049	total: 16.2s	remaining: 24.4s
2997:	learn: 0.0094405	total: 24.4s	remaining: 16.3s
3996:	learn: 0.0063288	total: 32.5s	remaining: 8.15s


20it [13:47, 41.38s/it]

4995:	learn: 0.0046748	total: 41s	remaining: 32.8ms
4999:	learn: 0.0046708	total: 41s	remaining: 0us
fold Loss 19:  0.090283136278654
---------------------------------------------------------------------------
Training Loss: 0.008880635416544571
Loss for sulfide:
0.07072439581147476 0.05140496493168944
AGG Loss: [0.14373991 0.08834108 0.16281245 0.19682514 0.01192014 0.15497703
 0.21814313 0.17247264 0.17649748 0.0707244 ]
AGG Loss mean: 0.13964534022091463


In [29]:
print(np.array(agg_loss).mean())

0.13964534022091463


In [30]:
test_preds = oof_test_df.iloc[:293]
train_preds = oof_train_df.iloc[766:]

In [31]:
display(test_preds)
display(train_preds)

,sample_id,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
766,NaN,0.003695,0.005422,0.008613,0.010144,0.000143,0.002188,0.013145,0.070672,0.008085,0.001035
767,NaN,0.032877,0.012275,0.001434,0.011949,0.000065,0.011461,0.021934,0.108794,0.015674,0.000803
768,NaN,0.836214,0.118500,0.001040,0.122598,0.000347,0.007584,0.765117,0.875549,0.078402,0.001345
769,NaN,0.009142,0.002678,0.047641,0.021531,0.000043,0.957916,0.024457,0.002406,0.887243,0.000438
770,NaN,0.002767,0.001679,0.014297,0.742778,0.000076,0.980744,0.975038,0.002381,0.003591,0.000137
...,...,...,...,...,...,...,...,...,...,...,...
1054,NaN,0.009490,0.002476,0.028519,0.005091,0.000079,0.004755,0.015281,0.067024,0.004724,0.000761
1055,NaN,0.009151,0.003652,0.002810,0.013612,0.002921,0.011045,0.015190,0.855772,0.003136,0.001561
1056,NaN,0.002547,0.008735,0.001046,0.010205,0.998697,0.002259,0.012129,0.003216,0.002684,0.000393
1057,NaN,0.002462,0.944072,0.017124,0.032702,0.000331,0.009359,0.046439,0.004991,0.002456,0.000534


,sample_id,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
766,NaN,0.004734,0.007877,0.011153,0.012424,0.000142,0.001978,0.016653,0.308473,0.008295,0.000838
767,NaN,0.053188,0.003731,0.001896,0.016599,0.000078,0.015483,0.037116,0.333673,0.021924,0.000577
768,NaN,0.395181,0.943676,0.001051,0.332658,0.000270,0.006950,0.322605,0.654329,0.313214,0.001412
769,NaN,0.019812,0.003369,0.175116,0.040647,0.000037,0.877976,0.029915,0.002030,0.450599,0.000392
770,NaN,0.002090,0.001867,0.016085,0.226213,0.000070,0.979685,0.967146,0.002144,0.004221,0.000100
...,...,...,...,...,...,...,...,...,...,...,...
1054,NaN,0.011826,0.003213,0.013582,0.005770,0.000072,0.005123,0.019616,0.152837,0.006672,0.000836
1055,NaN,0.010990,0.004168,0.003202,0.016475,0.004743,0.013901,0.018696,0.341544,0.003407,0.001389
1056,NaN,0.002861,0.013068,0.001119,0.011407,0.998236,0.002552,0.015114,0.003869,0.003427,0.000464
1057,NaN,0.004220,0.878433,0.025152,0.044000,0.000248,0.015184,0.078572,0.004713,0.003116,0.000529


In [32]:
agg_loss_train = []
for tar in targets.columns:
    loss = log_loss(val_labels[tar],train_preds[tar])
    print(tar, loss)
    agg_loss_train.append(loss)
print(np.array(agg_loss_train).mean())

basalt 0.1449698086850105
carbonate 0.09419039074169877
chloride 0.20126053320729925
iron_oxide 0.23934461102748875
oxalate 0.004095817905780798
oxychlorine 0.15823184075424335
phyllosilicate 0.22105009173340506
silicate 0.1607681896958841
sulfate 0.18553146869722542
sulfide 0.057331889723221584
0.14667746421712574


In [33]:
agg_loss_test = []
for tar in targets.columns:
    loss = log_loss(val_labels[tar],test_preds[tar])
    print(tar, loss)
    agg_loss_test.append(loss)
print(np.array(agg_loss_test).mean())

basalt 0.0248366137402643
carbonate 0.014295268356633492
chloride 0.03520467980169883
iron_oxide 0.05102750174348541
oxalate 0.0007399845772972207
oxychlorine 0.02741390861445691
phyllosilicate 0.057540429012452377
silicate 0.038926759970120746
sulfate 0.03125136433686359
sulfide 0.005341886475453164
0.028657839662872598


In [34]:
oof_test_df.sample_id = metadata[metadata.split != 'train'].sample_id
oof_train_df.sample_id = metadata[metadata.split != 'test'].sample_id

In [35]:
oof_train_df.to_csv(oof_train_path, index=False)
oof_test_df.to_csv(oof_test_path, index=False)